<a href="https://colab.research.google.com/github/ShamScripts/Medicine-Recommendation-System/blob/main/disease_prediction_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
# ==========================================
# Step 1: Import Libraries and Load Model
# ==========================================
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from fuzzywuzzy import process
from sklearn.ensemble import RandomForestClassifier

# Load trained Random Forest model
with open("model/RandomForest.pkl", "rb") as f:
    Rf = pickle.load(f)


In [68]:
# ==========================================
# Step 2: Load Datasets
# ==========================================
description = pd.read_csv("kaggle_dataset/description.csv")
precautions = pd.read_csv("kaggle_dataset/precautions_df.csv")
workout = pd.read_csv("kaggle_dataset/workout_df.csv")
symptoms_df = pd.read_csv("kaggle_dataset/symptoms_df.csv")
medications = pd.read_csv("kaggle_dataset/medications.csv")
diets = pd.read_csv("kaggle_dataset/diets.csv")
training_df = pd.read_csv("kaggle_dataset/Training.csv")


In [69]:
# ==========================================
# Step 3: Define Symptom and Disease Lists
# ==========================================
symptoms_list_processed = {col.replace('_', ' ').lower(): idx for idx, col in enumerate(training_df.columns[:-1])}
le = LabelEncoder()
le.fit(training_df['prognosis'])
diseases_list = dict(enumerate(le.classes_))


In [70]:
# ==========================================
# Step 4: Helper Functions
# ==========================================
def correct_spelling(symptom):
    matched, score = process.extractOne(symptom.lower(), list(symptoms_list_processed.keys()))
    return matched if score >= 80 else None

def predicted_value(patient_symptoms):
    vector = np.zeros(len(symptoms_list_processed))
    for symptom in patient_symptoms:
        if symptom in symptoms_list_processed:
            vector[symptoms_list_processed[symptom]] = 1
    prediction = Rf.predict([vector])[0]
    return diseases_list[prediction]

def information(predicted_dis):
    desc = description[description['Disease'] == predicted_dis]['Description'].values
    precaution = precautions[precautions['Disease'] == predicted_dis][['Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4']].values.tolist()
    meds = medications[medications['Disease'] == predicted_dis]['Medication'].values
    diet = diets[diets['Disease'] == predicted_dis]['Diet'].values
    workout_plan = workout[workout['disease'] == predicted_dis]['workout'].values
    return (
        desc[0] if len(desc) > 0 else "No description available.",
        precaution if len(precaution) > 0 else [["No precautions available."]],
        meds if len(meds) > 0 else ["No medication data."],
        diet if len(diet) > 0 else ["No diet data."],
        workout_plan if len(workout_plan) > 0 else ["No workout data."]
    )


In [72]:
# ==========================================
# Step 5: Testing
# ==========================================
user_input = input("Describe your symptoms (comma-separated): ")
input_symptoms = [s.strip() for s in user_input.split(',')]

corrected_symptoms = []
for symptom in input_symptoms:
    corrected = correct_spelling(symptom)
    if corrected:
        corrected_symptoms.append(corrected)
    else:
        print(f"Symptom '{symptom}' not recognized.")

if corrected_symptoms:
    predicted_disease = predicted_value(corrected_symptoms)
    disease_description, disease_precautions, disease_medications, disease_diet, disease_workout = information(predicted_disease)

    print("\nPredicted Disease:", predicted_disease)
    print("\nDescription:\n", disease_description)

    print("\nPrecautions:")
    for i, item in enumerate(disease_precautions[0], 1):
        print(f"{i}. {item}")

    print("\nMedications:")
    for i, med in enumerate(disease_medications, 1):
        print(f"{i}. {med}")

    print("\nWorkout:")
    for i, w in enumerate(disease_workout, 1):
        print(f"{i}. {w}")

    print("\nDiets:")
    for i, d in enumerate(disease_diet, 1):
        print(f"{i}. {d}")
else:
    print("\nNo valid symptoms found. Please try again.")

Describe your symptoms (comma-separated): fever

Predicted Disease: AIDS

Description:
 AIDS (Acquired Immunodeficiency Syndrome) is a disease caused by HIV that weakens the immune system.

Precautions:
1. avoid open cuts
2. wear ppe if possible
3. consult doctor
4. follow up

Medications:
1. ['Antiretroviral drugs', 'Protease inhibitors', 'Integrase inhibitors', 'Entry inhibitors', 'Fusion inhibitors']

Workout:
1. Follow a balanced and nutritious diet
2. Include lean proteins
3. Consume nutrient-rich foods
4. Stay hydrated
5. Include healthy fats
6. Avoid raw or undercooked foods
7. Limit sugary foods and beverages
8. Consume immune-boosting foods
9. Take prescribed supplements
10. Consult a healthcare professional

Diets:
1. ['Balanced Diet', 'Protein-rich foods', 'Fruits and vegetables', 'Whole grains', 'Healthy fats']


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
